# Consolidate Similarity Scores Across Multiple Runs

This notebook extracts and consolidates similarity scores from multiple run_summary.json files stored in Google Drive.

**Key Points:**
- Similarity scores are at the **instance level** (not instance-run level)
- They compare multiple runs of the same instance
- Located in: `multi_run_info.consistency_metrics.instance_reports`

**Setup:**
- Designed for Google Colab
- Mounts Google Drive to access data files
- Outputs saved to: `/content/drive/MyDrive/aisysevalteam-data/consistency_extraction/`

**Quick Start:**
1. Run all cells in order
2. Adjust `model_name` in Configuration cell for different models
3. CSV files and visualizations will be saved to the `consistency_extraction` subdirectory in Google Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import json
import pandas as pd
from pathlib import Path
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

## Configuration

**Update these variables to match your Google Drive structure:**
- `file_path`: Base directory in Google Drive where results are stored
- `model_name`: Specific model directory to analyze (e.g., 'google/gemini2.5pro')

In [ ]:
# Configuration
file_path = '/content/drive/MyDrive/aisysevalteam-data/'
# @TODO change this to extract per model
# available are: openai-gpt-5-nano, google-gemini2.5pro, qwen3-coder, openai/gpt-5
# model_name = 'google-gemini2.5pro'
model_name = 'openai/gpt-5'

# Construct base directory path
base_dir = Path(file_path) / model_name

# Alternative: Search all models in the base directory
# base_dir = Path(file_path)

print(f"Base data path: {file_path}")
print(f"Model name: {model_name}")
print(f"Searching in: {base_dir}")
print(f"Directory exists: {base_dir.exists()}")

# If directory doesn't exist, list what's available
if not base_dir.exists():
    print(f"\n⚠️ Directory not found!")
    print(f"\nAvailable directories in {file_path}:")
    base_path = Path(file_path)
    if base_path.exists():
        for item in sorted(base_path.iterdir()):
            if item.is_dir():
                print(f"  - {item.name}")
    else:
        print(f"⚠️ Base path {file_path} does not exist!")

## Find All run_summary.json Files

In [ ]:
# Find all run_summary.json files
run_summary_files = list(base_dir.glob("**/run_summary.json"))

print(f"Found {len(run_summary_files)} run_summary.json files:\n")
for f in sorted(run_summary_files):
    print(f"  {f.relative_to(base_dir.parent)}")

## Extract Similarity Scores by Instance

In [ ]:
# Dictionary to store all data by instance_id
# Structure: {instance_id: [record1, record2, ...]}
instances_data = defaultdict(list)

for summary_file in run_summary_files:
    try:
        with open(summary_file, 'r') as f:
            data = json.load(f)

        # Extract run metadata
        run_info = data.get('run_info', {})
        model_name_from_file = run_info.get('model_name', 'unknown')
        timestamp = run_info.get('timestamp', 'unknown')
        num_runs_configured = run_info.get('num_runs_configured', 1)

        # Calculate relative path for run_dir
        try:
            run_dir_rel = str(summary_file.parent.relative_to(base_dir))
        except ValueError:
            # If relative_to fails, use the full path
            run_dir_rel = str(summary_file.parent)

        # Extract instance reports from multi_run_info
        multi_run_info = data.get('multi_run_info', {})
        consistency_metrics = multi_run_info.get('consistency_metrics', {})

        # Handle both completed consistency checks and skipped ones
        if isinstance(consistency_metrics, dict):
            instance_reports = consistency_metrics.get('instance_reports', [])
        else:
            instance_reports = []

        # Process each instance report
        for report in instance_reports:
            instance_id = report.get('instance_id')
            if not instance_id:
                continue

            # Extract patch consistency metrics
            patch_consistency = report.get('patch_consistency', {})
            evaluation_consistency = report.get('evaluation_consistency', {})
            security_consistency = report.get('security_consistency', {})

            # Create record
            record = {
                'instance_id': instance_id,
                'model_name': model_name_from_file,
                'timestamp': timestamp,
                'run_dir': run_dir_rel,
                'num_runs': report.get('num_runs', 1),
                'num_runs_configured': num_runs_configured,

                # Similarity scores (0.0-1.0)
                'avg_ast_similarity': patch_consistency.get('avg_ast_similarity'),
                'avg_text_similarity': patch_consistency.get('avg_text_similarity'),
                'avg_hybrid_similarity': patch_consistency.get('avg_hybrid_similarity'),

                # Other patch metrics
                'exact_match_rate': patch_consistency.get('exact_match_rate'),
                'num_unique_patches': patch_consistency.get('num_unique_patches'),
                'confidence_percent': patch_consistency.get('confidence_percent'),
                'normalized_confidence_percent': patch_consistency.get('normalized_confidence_percent'),
                'agreement_percent': patch_consistency.get('agreement_percent'),
                'line_count_variance': patch_consistency.get('line_count_variance'),

                # Consistency scores
                'overall_consistency_score': report.get('overall_consistency_score'),
                'consistency_grade': report.get('consistency_grade'),

                # Component scores
                'patch_score': report.get('component_scores', {}).get('patch_score'),
                'evaluation_score': report.get('component_scores', {}).get('evaluation_score'),
                'security_score': report.get('component_scores', {}).get('security_score'),

                # Evaluation consistency
                'all_resolved': evaluation_consistency.get('all_resolved'),
                'all_failed': evaluation_consistency.get('all_failed'),
                'resolution_rate': evaluation_consistency.get('resolution_rate'),
                'num_evaluations': evaluation_consistency.get('num_evaluations'),

                # Security consistency
                'risk_level_mode': security_consistency.get('risk_level_mode'),
                'security_score_variance': security_consistency.get('score_variance'),
                'num_scans': security_consistency.get('num_scans'),
                'avg_security_score': security_consistency.get('avg_score'),
            }

            instances_data[instance_id].append(record)

    except Exception as e:
        print(f"Error processing {summary_file}: {e}")
        continue

print(f"\nExtracted data for {len(instances_data)} unique instances")
print(f"Total records: {sum(len(records) for records in instances_data.values())}")

## Create Consolidated DataFrame

In [ ]:
# Flatten the data into a list of records
all_records = []
for instance_id, records in instances_data.items():
    all_records.extend(records)

# Create DataFrame
df = pd.DataFrame(all_records)

# Sort by instance_id and timestamp
df = df.sort_values(['instance_id', 'timestamp'])

print(f"Created DataFrame with {len(df)} rows and {len(df.columns)} columns")
print(f"\nColumns: {list(df.columns)}")

df.head(10)

## Data Dictionary

The extracted dataset includes the following fields:

### Metadata
- **instance_id**: SWE-bench instance identifier
- **model_name**: AI model used
- **timestamp**: Run timestamp
- **run_dir**: Directory containing the run data
- **num_runs**: Actual number of runs performed
- **num_runs_configured**: Number of runs configured

### Similarity Scores (0.0-1.0, higher = more similar)
- **avg_ast_similarity**: Average structural code similarity (AST-based)
- **avg_text_similarity**: Average surface-level text similarity
- **avg_hybrid_similarity**: Weighted combination of AST and text similarity

### Patch Consistency Metrics
- **exact_match_rate**: Fraction of patches that are byte-for-byte identical
- **num_unique_patches**: Number of distinct patches generated
- **confidence_percent**: Main consistency metric (0-100)
- **normalized_confidence_percent**: Rescaled confidence metric (0-100)
- **agreement_percent**: Percentage of pairs above similarity threshold (0-100)
- **line_count_variance**: Variation in patch sizes

### Overall Consistency
- **overall_consistency_score**: Weighted score combining all metrics (0-100)
- **consistency_grade**: EXCELLENT / GOOD / FAIR / POOR

### Component Scores (0-100)
- **patch_score**: Patch consistency component (40% weight)
- **evaluation_score**: Evaluation consistency component (40% weight)
- **security_score**: Security consistency component (20% weight)

### Evaluation Consistency
- **all_resolved**: Whether all runs resolved the issue
- **all_failed**: Whether all runs failed the evaluation
- **resolution_rate**: Fraction of runs that passed (0.0-1.0)
- **num_evaluations**: Number of evaluations performed

### Security Consistency
- **risk_level_mode**: Most common security risk level (NONE/LOW/MEDIUM/HIGH)
- **security_score_variance**: Variance in security scores
- **num_scans**: Number of security scans performed
- **avg_security_score**: Average security risk score across runs

## Summary Statistics

In [ ]:
print("=" * 80)
print("SIMILARITY SCORES SUMMARY")
print("=" * 80)

similarity_cols = ['avg_ast_similarity', 'avg_text_similarity', 'avg_hybrid_similarity']
print("\nSimilarity Scores Statistics (0.0 = very different, 1.0 = identical):")
print(df[similarity_cols].describe())

print("\n" + "=" * 80)
print("CONSISTENCY GRADES DISTRIBUTION")
print("=" * 80)
print(df['consistency_grade'].value_counts())

print("\n" + "=" * 80)
print("CONSISTENCY SCORES BY GRADE")
print("=" * 80)
print(df.groupby('consistency_grade')['overall_consistency_score'].agg(['mean', 'min', 'max', 'count']))

## View Specific Instances

In [ ]:
# Show instances with multiple runs across different timestamps
instance_counts = df.groupby('instance_id').size().sort_values(ascending=False)
print("Instances with multiple consistency reports:")
print(instance_counts[instance_counts > 1])

# Pick an instance to examine in detail
if len(instance_counts) > 0:
    example_instance = instance_counts.index[0]
    print(f"\n{'='*80}")
    print(f"Detailed view of: {example_instance}")
    print("="*80)

    cols_to_show = [
        'timestamp', 'num_runs',
        'avg_ast_similarity', 'avg_text_similarity', 'avg_hybrid_similarity',
        'overall_consistency_score', 'consistency_grade'
    ]

    display(df[df['instance_id'] == example_instance][cols_to_show])

## Filter by Consistency Grade

In [ ]:
# Show instances with EXCELLENT consistency
excellent = df[df['consistency_grade'] == 'EXCELLENT']
print(f"Instances with EXCELLENT consistency: {len(excellent)}")
if len(excellent) > 0:
    display(excellent[['instance_id', 'timestamp', 'avg_ast_similarity',
                       'avg_text_similarity', 'overall_consistency_score']].head(10))

# Show instances with POOR consistency
poor = df[df['consistency_grade'] == 'POOR']
print(f"\nInstances with POOR consistency: {len(poor)}")
if len(poor) > 0:
    display(poor[['instance_id', 'timestamp', 'avg_ast_similarity',
                  'avg_text_similarity', 'overall_consistency_score']].head(10))

In [ ]:
# Create output directory
output_dir = Path(file_path) / "consistency_extraction"
output_dir.mkdir(parents=True, exist_ok=True)
print(f"Output directory: {output_dir}\n")

# Export full dataset
output_file = output_dir / f'consolidated_similarity_scores_{model_name.replace("/", "_")}.csv'
df.to_csv(output_file, index=False)
print(f"✅ Exported full dataset to: {output_file}")

# Export summary by instance (latest run only)
summary_file = output_dir / f'similarity_scores_summary_{model_name.replace("/", "_")}.csv'
latest_df = df.sort_values('timestamp').groupby('instance_id').last().reset_index()
latest_df.to_csv(summary_file, index=False)
print(f"✅ Exported summary (latest run per instance) to: {summary_file}")

print(f"\n📊 Full dataset: {len(df)} records")
print(f"📊 Summary dataset: {len(latest_df)} records (one per instance)")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Plot 1: Distribution of similarity scores
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

df['avg_ast_similarity'].hist(bins=20, ax=axes[0], edgecolor='black')
axes[0].set_title('AST Similarity Distribution')
axes[0].set_xlabel('Similarity Score')
axes[0].set_ylabel('Frequency')

df['avg_text_similarity'].hist(bins=20, ax=axes[1], edgecolor='black')
axes[1].set_title('Text Similarity Distribution')
axes[1].set_xlabel('Similarity Score')

df['avg_hybrid_similarity'].hist(bins=20, ax=axes[2], edgecolor='black')
axes[2].set_title('Hybrid Similarity Distribution')
axes[2].set_xlabel('Similarity Score')

plt.tight_layout()
plot1_file = Path(file_path) / f'similarity_distributions_{model_name.replace("/", "_")}.png'
plt.savefig(plot1_file, dpi=150, bbox_inches='tight')
plt.show()

# Plot 2: Consistency scores by grade
fig, ax = plt.subplots(figsize=(10, 6))
df.boxplot(column='overall_consistency_score', by='consistency_grade', ax=ax)
plt.title('Consistency Scores by Grade')
plt.suptitle('')  # Remove default title
plt.xlabel('Consistency Grade')
plt.ylabel('Overall Consistency Score')
plot2_file = Path(file_path) / f'consistency_by_grade_{model_name.replace("/", "_")}.png'
plt.savefig(plot2_file, dpi=150, bbox_inches='tight')
plt.show()

print("✅ Saved visualizations:")
print(f"  - {plot1_file}")
print(f"  - {plot2_file}")

In [ ]:
# Analyze correlation between different metrics
correlation_cols = [
    'avg_ast_similarity', 'avg_text_similarity', 'avg_hybrid_similarity',
    'exact_match_rate', 'confidence_percent', 'overall_consistency_score'
]

correlation_matrix = df[correlation_cols].corr()

# Use the output directory
output_dir = Path(file_path) / "consistency_extraction"

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=1)
plt.title('Correlation Between Consistency Metrics')
plt.tight_layout()
heatmap_file = output_dir / f'correlation_heatmap_{model_name.replace("/", "_")}.png'
plt.savefig(heatmap_file, dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Saved {heatmap_file}")

## Notes

**Configuration:**
- All outputs are saved to Google Drive: `/content/drive/MyDrive/aisysevalteam-data/consistency_extraction/`
- Files are named with the model name to avoid conflicts: `consolidated_similarity_scores_google_gemini2.5pro.csv`
- Update the `model_name` variable in the Configuration cell to analyze different models

**File Outputs:**
All files are saved to the `consistency_extraction` subdirectory:
1. CSV files with similarity scores (full and summary)
2. Visualization PNGs (distributions, boxplot, correlation heatmap)

**Output Structure:**
```
/content/drive/MyDrive/aisysevalteam-data/
└── consistency_extraction/
    ├── consolidated_similarity_scores_google_gemini2.5pro.csv
    ├── similarity_scores_summary_google_gemini2.5pro.csv
    ├── similarity_distributions_google_gemini2.5pro.png
    ├── consistency_by_grade_google_gemini2.5pro.png
    └── correlation_heatmap_google_gemini2.5pro.png
```

**To analyze a different model:**
Simply change the `model_name` variable in the Configuration cell:
```python
model_name = 'anthropic/claude-sonnet-4'
```

**To analyze all models:**
Comment out the model-specific path and use the base directory:
```python
# base_dir = Path(file_path) / model_name
base_dir = Path(file_path)
```

## Notes

**Configuration:**
- All outputs are saved to Google Drive: `/content/drive/MyDrive/aisysevalteam-data/`
- Files are named with the model name to avoid conflicts: `consolidated_similarity_scores_google_gemini2.5pro.csv`
- Update the `model_name` variable in the Configuration cell to analyze different models

**File Outputs:**
1. CSV files with similarity scores (full and summary)
2. Visualization PNGs (distributions, boxplot, correlation heatmap)

**To analyze a different model:**
Simply change the `model_name` variable in the Configuration cell:
```python
model_name = 'anthropic/claude-sonnet-4'
```

**To analyze all models:**
Comment out the model-specific path and use the base directory:
```python
# base_dir = Path(file_path) / model_name
base_dir = Path(file_path)
```